In [ ]:
import os
import lxml
import pytz
from lxml import etree
import datetime
from collections import defaultdict
import green_button
import solar_data

tz = pytz.timezone('US/Pacific')
mindate = datetime.date(2020, 6, 1)
maxdate = datetime.date(2020, 11, 6)
include_solar = True
include_meter = True


In [ ]:
solardatadict = solar_data.load_solar_data_from_json(tz)

In [ ]:
import plotly.graph_objects as go
import datetime
import numpy as np
import ipywidgets as ipw

def run(contents, mindate, maxdate):
    datadict, dates = green_button.load_sce_data_from_xml(contents, tz)
    ndates = (maxdate - mindate).days + 1
    dates = [mindate + datetime.timedelta(days=i) for i in range(ndates)]
    tstimes = [f'{i//4:02}:{15*(i%4):02}' for i in range(96)]
    vals = np.zeros(shape=(96, ndates))
    multiplier = 1.

    if include_meter:
        for k, v in datadict.items():
            tsdate, tstime = k
            dateind = (tsdate - mindate).days
            if 0 <= dateind < ndates: 
                vals[tstime, dateind] += v * multiplier

    if include_solar:
        for k, v in solardatadict.items():
            tsdate, tstime = k
            dateind = (tsdate - mindate).days
            if 0 <= dateind < ndates:
                vals[tstime, dateind] += v * multiplier

    fig = go.Figure(data=go.Heatmap(
            z=vals,
            x=dates,
            y=tstimes,
            #colorscale='Viridis'
        )
    )

    fig.update_layout(
        title='Power usage',
        xaxis_nticks=36,
        #autosize=False,
        #width=5000,
        #height=2000,
        #margin=dict(
        #    l=50,
        #    r=50,
        #    b=100,
        #    t=100,
        #    pad=4
        #),
    )

    fig.show()


In [ ]:
from IPython.display import display, clear_output
startipw = ipw.DatePicker()
endipw = ipw.DatePicker()
runbtn = ipw.Button(description="run", layout=ipw.Layout(width="45px"))
uploader = ipw.FileUpload(accept='.xml', multiple=True)
usl = ipw.Label("Upload SCE usage data xml here: ")

def on_click(btn):
    clear_output()
    display(show_widget())
    s = startipw.value
    e = endipw.value
    if s is not None and e is not None:
        run([v['content'] for v in uploader.value.values()], s, e)
        
runbtn.on_click(on_click)

def show_widget():
    return ipw.VBox((ipw.HBox((usl, uploader)), startipw, endipw, runbtn))
    
show_widget()